# Test Matching Insee/ETS

## INSEE

- https://s3.console.aws.amazon.com/s3/object/calfdata/INSEE/Stock/ETS/
        - INSEE/Stock/ETS/StockEtablissement_utf8.csv
        
```
['siren', 'siret']
```

## INPI

- https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/Stock_processed/
    - INPI/TC_1/Stock_processed/initial_ETS.gz
    - INPI/TC_1/Stock_processed/initial_ETS.json
    
Colonnes test:

```
["Siren","Date_Immatriculation", "Date_Clôture", "Date_Greffe"]
```

## Sauvegarde

* La liste des SIREN matchés sera sauvegardée selon leur nature et origine
  * nature → ACTES/COMPTES/ETS/etc
  * origine → initial/partiel/new/evt

Les matchés seront sauvegardé dans calfdata/SIRETISATION/matche/ au format suivant:

* insee_nature_origine_matche.gz
    * ex: insee_pm_initial_matche.gz
    
    

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/
* Infogreffe
  * https://www.infogreffe.fr/


In [ ]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
#import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [ ]:
#instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
#bucket = 'calfdata'

In [ ]:
# instanciate AWS connection
#AWS_connection = aws.aws_instantiate(instance_aws, bucket)

## Valeurs communes dans INSEE/INPI ETS

| INSEE                     | INPI         |                         |
|---------------------------|--------------|-------------------------|
| Var                       | Var          | comment                 |
| siren                     | Siren        |                         |
| codePostalEtablissement   | Code_Postal  | Si Type == 'principale' |
| codePostal2Etablissement  | Code_Postal  | Si Type == 'secondaire' |
| codeCommuneEtablissement  | Code_Commune | Si Type == 'principale' |
| codeCommune2Etablissement | Code_Commune | Si Type == 'secondaire' |
|                           |              |                         |
|                           |              |                         |
|                           |              |                         |

In [ ]:
#insee = AWS_connection.url_instance_bucket(path_file = 'INSEE/Stock/ETS/StockEtablissement_utf8.csv')
#ets = AWS_connection.url_instance_bucket(path_file = 'INPI/TC_1/Stock_processed/initial_ETS.gz')
#ets

In [ ]:
insee = r"\temp_local\StockEtablissement_utf8.csv"
ets = r"\temp_local\initial_ETS.gz"

## Matching établissement principal

Ici, on filtre les variables communes pour l'INSEE & INPI établissements secondaires.

### Candidats

**INSEE**

https://www.sirene.fr/sirene/public/static/liste-variables

- numeroVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/numeroVoieEtablissement
- indiceRepetitionEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/indiceRepetitionEtablissement
- typeVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/typeVoieEtablissement
- libelleVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleVoieEtablissement
- complementAdresseEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/complementAdresseEtablissement
- codeCommuneEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codeCommuneEtablissement
- libelleCommuneEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCommuneEtablissement
- codePostalEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codePostalEtablissement
- codeCedexEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codeCedexEtablissement
- libelleCedexEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCedexEtablissement
- distributionSpecialeEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/distributionSpecialeEtablissement
- libelleCommuneEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCommuneEtrangerEtablissement
- codePaysEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codePaysEtrangerEtablissement
- libellePaysEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libellePaysEtrangerEtablissement

**INPI**

- Adresse_Ligne1/Adresse_Ligne2/Adresse_Ligne3: Selon les greffes, l’adresse (n°+ voie) sera présente soit en ligne1 adresse, soit en ligne2 adresse.
Toutes les lignes d’adresse ne sont pas nécessairement renseignées.


### Process

Calculer le nombre de `nan` dans les colonnes de matching

In [ ]:
# load data into dataframes
data_insee_ = dd.read_csv(insee,
                          usecols=['siren',
                                   'siret',
                                   "numeroVoieEtablissement",
                                   "indiceRepetitionEtablissement",
                                   "typeVoieEtablissement",
                                   "libelleVoieEtablissement",
                                   "complementAdresseEtablissement",
                                   "codeCommuneEtablissement",
                                   "libelleCommuneEtablissement",
                                   "codePostalEtablissement",
                                   "codeCedexEtablissement",
                                   "libelleCedexEtablissement",
                                   "distributionSpecialeEtablissement",
                                   "libelleCommuneEtrangerEtablissement",
                                   "codePaysEtrangerEtablissement",
                                   "libellePaysEtrangerEtablissement"
                                   ],
                          dtype={'siren': 'object',
                                 'siret': 'object',
                                 "numeroVoieEtablissement":'object',
                                   "indiceRepetitionEtablissement":'object',
                                   "typeVoieEtablissement":'object',
                                   "libelleVoieEtablissement":'object',
                                   "complementAdresseEtablissement":'object',
                                   "codeCommuneEtablissement":'object',
                                   "libelleCommuneEtablissement":'object',
                                   "codePostalEtablissement":'object',
                                   "codeCedexEtablissement":'object',
                                   "libelleCedexEtablissement":'object',
                                   "distributionSpecialeEtablissement":'object',
                                   "libelleCommuneEtrangerEtablissement":'object',
                                   "codePaysEtrangerEtablissement":'object',
                                   "libellePaysEtrangerEtablissement":'object'
                                 }
                          )

data_ets_ = (dd.read_csv(ets,
                         usecols=[
                             'Type',
                             'Siren',
                             'Code_Postal',
                             'Code_Commune',
                             'Adresse_Ligne1',
                             'Adresse_Ligne2',
                             'Adresse_Ligne3',
                             'Ville',
                             'Pays'
                         ],
                         dtype={
                             'Type': 'object',
                             'Siren': 'object',
                             'Code_Postal': 'object',
                             'Code_Commune': 'object',
                             'Adresse_Ligne1': 'object',
                             'Adresse_Ligne2': 'object',
                             'Adresse_Ligne3': 'object',
                             'Ville':'object',
                             'Pays':'object'
                         },
                         compression='gzip',
                         blocksize=None,
                         low_memory=False
                         )
             .compute()
             .rename(columns={"Siren": "siren"})
             .loc[lambda x: x['Type'].isin(['PRI'])]
             )

In [ ]:
data_ets_.shape

In [ ]:
siren_inpi = data_ets_['siren'].drop_duplicates()
len(siren_inpi)

In [ ]:
subset_insee = data_insee_.compute().loc[lambda x: x['siren'].isin(siren_inpi)]
subset_insee.shape

Siren INPI mais pas INSEE -> Cette entreprise a exercé son droit d'opposition auprès de l'INSEE. Ses données ne peuvent pas être diffusées publiquement.

In [ ]:
siren_to_remove = siren_inpi.loc[lambda x : ~x.isin(subset_insee['siren'])]
len(siren_to_remove)

In [ ]:
df_siren_to_find = data_ets_.loc[lambda x:
                                 (~x['siren'].isin(siren_to_remove))
                                 #| (~x['Adresse_Ligne1'].isin([np.nan]))
                                 #& (~x['Adresse_Ligne2'].isin([np.nan]))
                                 #& (~x['Adresse_Ligne3'].isin([np.nan]))
                                 #& (~x['Code_Postal'].isin([np.nan]))
                                 #& (~x['Ville'].isin([np.nan]))
                                 #& (~x['Code_Commune'].isin([np.nan]))
                                 ]
len(siren_to_find)

### Nan variables matching 

on exclue les variables avec que des nan dans les variables candidates

In [ ]:
import numpy as np
siren_fullna = df_siren_to_find.loc[lambda x:
                      (x['Adresse_Ligne1'].isin([np.nan]))
                     & (x['Adresse_Ligne2'].isin([np.nan]))
                     & (x['Adresse_Ligne3'].isin([np.nan]))
                     & (x['Code_Postal'].isin([np.nan]))
                     & (x['Ville'].isin([np.nan]))
                     & (x['Code_Commune'].isin([np.nan]))
                     ]['siren']

In [ ]:
df_siren_to_find = df_siren_to_find.loc[lambda x:
                                 (~x['siren'].isin(siren_fullna))
                                 ]
len(siren_fullna)

In [ ]:
df_siren_to_find.shape

Quick stat

In [ ]:
(subset_insee
 .groupby('siren')['siren']
 .count()
 .rename('count')
 .reset_index()
 .groupby('count')['count']
 .count()
 .reset_index(name = 'total_count')
 .set_index('count')
 #.compute()
 .assign(pct=lambda x: x/x.sum())
 .iloc[:10,:]
 .style
 .format('{:,.2%}', subset=['pct'])
 )

In [ ]:
df_siren_to_find['Type'].unique()

## Missing values

In [ ]:
data_insee_.isna().sum().compute()

In [ ]:
df_siren_to_find.isna().sum().sort_values()

In [ ]:
df_siren_to_find.shape

In [ ]:
def mergeInseeEtbs(dataETBS, left_on, right_on):
    """
    """
    
    var_insee = ['siret',
             "numeroVoieEtablissement",
             "indiceRepetitionEtablissement",
             "typeVoieEtablissement",
             "libelleVoieEtablissement",
             "complementAdresseEtablissement",
             "codeCommuneEtablissement",
             "libelleCommuneEtablissement",
             "codePostalEtablissement",
             "codeCedexEtablissement",
             "libelleCedexEtablissement",
             "distributionSpecialeEtablissement",
             "libelleCommuneEtrangerEtablissement",
             "codePaysEtrangerEtablissement",
             "libellePaysEtrangerEtablissement",
             '_merge'
             ]
    
    data_merged_1 = (subset_insee
                     .merge(
                         dataETBS,
                         how='right',
                         left_on=left_on,
                         right_on=right_on,
                         indicator=True)
                     # .compute()
                     )
    ### remove INSEE variables in data_merge_1
    unmatched_1 = (data_merged_1
             .loc[lambda x: x['_merge'].isin(["right_only"])]
             .drop(columns  = var_insee)
            )
    
    ### compute count merged:unmerged
    count_ = data_merged_1.groupby('_merge')["_merge"].count()
    
    dic_ = {
        'unmerged':unmatched_1, 
        'count_':count_
    }
    
    return dic_

## Merging

### Processus
 
- Merge sur siren & codePostalEtablissement

Missing values:

- codePostalEtablissement:168253
- Code_Postal: 20734

In [ ]:
m1 = mergeInseeEtbs(dataETBS = df_siren_to_find,
               left_on = ["siren", 'codePostalEtablissement'],
               right_on = ['siren', 'Code_Postal'])

In [ ]:
m1['count_']

## Merging

### Processus
 
- Merge sur siren & codeCommuneEtablissement

Missing values:

- codeCommuneEtablissement:168253
- Code_Commune: 99457

In [ ]:
m2 = mergeInseeEtbs(dataETBS = m1['unmerged'],
               left_on = ["siren", 'codeCommuneEtablissement'],
               right_on =['siren', 'Code_Commune']
                   )

In [ ]:
m2['count_']

In [ ]:
(m2['count_'][1]/3017128)  * 100

In [ ]:
m2['unmerged'].head()

### Processus 3


In [ ]:
m2['unmerged'].isna().sum()

In [ ]:
m2['unmerged']

In [ ]:
subset_insee.loc[lambda x: x['siren'].isin(["420281784"])]

In [ ]:
temp.assign(url = lambda x :
            'https://data.inpi.fr/entreprises/' + x["Siren"] )